In [2]:
import os
import torch
import torch.nn as nn
import importlib
import glob
import numpy as np
import motifdata as md
from pkg_resources import resource_filename
from eugene import models
from eugene import settings
settings.logging_dir = "/cellar/users/aklie/projects/ML4GLand/EUGENe_paper/logs/revision/jores21"
settings.config_dir = "/cellar/users/aklie/projects/ML4GLand/EUGENe_paper/configs/jores21"

In [3]:
def describe_motif_set(motif_set):
    print(f"# motifs: {len(motif_set)}")
    print(f"Alphabet: {motif_set.alphabet}")
    print(f"Version: {motif_set.version}")
    print(f"Strands: {motif_set.strands}")
    print(f"Background: {motif_set.background}")
    print(f"Background source: {motif_set.background_source}")

In [4]:
def print_motifs(motif_set, n=None):
    for motif in motif_set:
        print(f"Identifer: {motif.identifier}")
        print(f"Name: {motif.name}")
        print(f"Consensus: {motif.consensus}")
        print(f"PFM: {motif.pfm} with shape {motif.pfm.shape}")
        print()
        if n is not None:
            n -= 1
            if n == 0:
                break

In [5]:
def list_available_layers(
    model: nn.Module,
    key_word = None
) -> list:
    """List the available layers in a model
    
    Parameters
    ----------
    model : torch.nn.Module
        The model to list the layers of
    key_word : str, optional
        A key word to filter the layers by, by default None
    
    Returns
    -------
    list
        A list of the available layers in the model
    """
    layers = sorted([k for k in dict([*model.named_modules()])])
    if key_word is not None:
        layers = [layer for layer in layers if key_word in layer]
    return layers

In [6]:
def get_layer(
    model: nn.Module,
    layer_name: str
) -> nn.Module:
    """Get a layer from a model by name. Note that this will only work for
    named modules. If the model has unnamed modules, TODO

    Parameters
    ----------
    model : torch.nn.Module
        The model to get the layer from
    layer_name : str
        The name of the layer to get

    Returns
    -------
    torch.nn.Module
        The layer from the model
    """
    return dict([*model.named_modules()])[layer_name]

# Convert a kernel to MEME

In [18]:
leaf_model_file = glob.glob(os.path.join(settings.logging_dir, "cnn", "leaf_trial_3", "checkpoints", "*"))[0]
model = models.load_config(config_path="cnn.yaml")
leaf_model = models.SequenceModule.load_from_checkpoint(leaf_model_file, arch=model.arch)

[rank: 0] Global seed set to 3


In [19]:
list_available_layers(leaf_model, key_word="conv")

['arch.conv1d_tower',
 'arch.conv1d_tower.layers',
 'arch.conv1d_tower.layers.0',
 'arch.conv1d_tower.layers.1',
 'arch.conv1d_tower.layers.10',
 'arch.conv1d_tower.layers.11',
 'arch.conv1d_tower.layers.12',
 'arch.conv1d_tower.layers.13',
 'arch.conv1d_tower.layers.14',
 'arch.conv1d_tower.layers.2',
 'arch.conv1d_tower.layers.3',
 'arch.conv1d_tower.layers.4',
 'arch.conv1d_tower.layers.5',
 'arch.conv1d_tower.layers.6',
 'arch.conv1d_tower.layers.7',
 'arch.conv1d_tower.layers.8',
 'arch.conv1d_tower.layers.9']

In [20]:
# Grab the first convolutional layer and set kernels < 0 to 0
layer = get_layer(leaf_model, "arch.conv1d_tower.layers.0")
kernels = layer.weight.data.numpy()
kernels[kernels < 0] = 0

In [21]:
motif_set = md.from_kernel(kernels, bg = {"A": 0.25, "C": 0.25, "G": 0.25, "T": 0.25})
describe_motif_set(motif_set)
print()
print_motifs(motif_set, n=1)

# motifs: 256
Alphabet: ACGT
Version: 5
Strands: + -
Background: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
Background source: None

Identifer: filter_0
Name: filter_0
Consensus: TCTCTATAAATAC
PFM: [[0.3872321  1.4875679  0.4326479  1.5474236 ]
 [0.5810361  1.6907147  0.6329294  0.92154586]
 [0.92491764 1.0179404  0.8485242  1.0865945 ]
 [0.541581   2.6780574  0.23881885 0.3438842 ]
 [0.         0.         0.02326916 4.0359554 ]
 [3.8641582  0.         0.         0.12367285]
 [0.         0.         0.         4.0058208 ]
 [3.9876742  0.         0.06564526 0.        ]
 [2.4759955  0.         0.10032247 1.3471979 ]
 [3.86505    0.         0.06999157 0.        ]
 [1.2255843  0.07950763 0.13378221 2.3990386 ]
 [2.8052242  0.22689693 0.4605895  0.36558354]
 [0.40350372 1.7106136  1.2255125  0.5111682 ]] with shape (13, 4)



In [22]:
md.write_meme(motif_set, resource_filename('motifdata', 'resources/filters_out.meme'))

Saved pfm in MEME format as: /cellar/users/aklie/projects/ML4GLand/MotifData/motifdata/resources/filters_out.meme


# Load MotifSet into model kernels

In [62]:
#motif_set = md.read_homer(resource_filename('motifdata', 'resources/sample.motifs'))
#motif_set = md.read_meme("/cellar/users/aklie/data/ml4gland/use_cases/jores21/paper_github/CPEs.meme")
motif_set = md.read_meme("/cellar/users/aklie/data/ml4gland/use_cases/jores21/paper_github/TF-clusters.meme")
describe_motif_set(motif_set)
print()
print_motifs(motif_set, n=1)

# motifs: 72
Alphabet: ACGT
Version: 5
Strands: + -
Background: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
Background source: None

Identifer: TF-cluster_1
Name: NAC
Consensus: TTACGTGTAGAACAAG
PFM: [[0.1949128  0.01640289 0.00895095 0.7797334 ]
 [0.2782767  0.00825623 0.2022142  0.511253  ]
 [0.4608514  0.1265213  0.3093912  0.1032361 ]
 [0.02296122 0.9709426  0.001743   0.00435315]
 [0.00325958 0.01182932 0.5375714  0.4473397 ]
 [0.00417949 0.00398073 0.01938412 0.9724556 ]
 [0.2775764  0.06676622 0.6315887  0.02406878]
 [0.2904112  0.186561   0.1394217  0.383606  ]
 [0.3518847  0.1558543  0.246462   0.245799  ]
 [0.2788753  0.1914388  0.3083355  0.2213506 ]
 [0.3565019  0.2176151  0.1845846  0.2412985 ]
 [0.4840646  0.09719634 0.1803039  0.2384354 ]
 [0.06016895 0.5575891  0.2264818  0.1557602 ]
 [0.944828   0.01806365 0.00485082 0.03225751]
 [0.6672574  0.2912842  0.02046174 0.02099667]
 [0.01196582 0.00227141 0.9621388  0.02362408]] with shape (16, 4)



In [60]:
model = models.load_config(config_path="cnn.yaml")
layer_name = "arch.conv1d_tower.layers.0"
layer = get_layer(model, layer_name)
pre_init_layer_weights = layer.weight.data.numpy().copy()
pwms = md.to_kernel(motif_set, tensor=layer.weight.data, convert_to_pwm=False, divide_by_bg=False, motif_align="right", kernel_align="right")
layer.weight = nn.Parameter(pwms)
layer_weights = get_layer(model, layer_name).weight.data.numpy()
pre_init_layer_weights[5].T, layer_weights[5].T

(array([[ 0.12173441, -0.08135366,  0.05647704, -0.0386282 ],
        [-0.02299144,  0.01623187,  0.11310549,  0.0295976 ],
        [ 0.09796639, -0.06018299, -0.01161144, -0.01280014],
        [-0.00947437, -0.10643078,  0.0982954 , -0.06520253],
        [ 0.0076355 , -0.11207107,  0.01673558, -0.12620355],
        [-0.0472234 ,  0.01564597,  0.08229609, -0.1126551 ],
        [-0.1283058 , -0.0463211 ,  0.10362144,  0.06945334],
        [ 0.11920424, -0.07438475,  0.12470192,  0.00858062],
        [-0.12152293,  0.1052118 ,  0.03757446, -0.02479482],
        [ 0.00618484, -0.00333846,  0.01732013,  0.03577886],
        [ 0.02141361,  0.11727677,  0.04337198, -0.03345665],
        [-0.03767323,  0.08713868, -0.07901561, -0.13641895],
        [ 0.0264579 ,  0.12546062,  0.10012502,  0.1140984 ]],
       dtype=float32),
 array([[ 1.21734411e-01, -8.13536569e-02,  5.64770401e-02,
         -3.86282019e-02],
        [-2.29914431e-02,  1.62318721e-02,  1.13105491e-01,
          2.95976046e-0

In [61]:
# Count the number of kernels that are the same before and after. Use close enough to account for floating point errors
num_close = 0
for i in range(len(pre_init_layer_weights)):
    if np.allclose(pre_init_layer_weights[i], layer_weights[i]):
        num_close += 1
print(f"Number of kernels that are the same: {num_close}")

Number of kernels that are the same: 184


# DONE!

---

# Scratch